In [1]:
import os
import dask
import dask.dataframe as dd

df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={'TailNum': str,
                        'CRSElapsedTime': float,
                        'Cancelled': bool})



crs_dep_time = df.CRSDepTime.head(10)
crs_dep_time

0    1055
1    1055
2    1055
3    1729
4    1729
5    1729
6    1729
7    1729
8    1729
9    1729
Name: CRSDepTime, dtype: int64

In [2]:

import pandas as pd

# Get the first 10 dates to complement our `crs_dep_time`
date = df.Date.head(10)

# Get hours as an integer, convert to a timedelta
hours = crs_dep_time // 100
hours_timedelta = pd.to_timedelta(hours, unit='h')

# Get minutes as an integer, convert to a timedelta
minutes = crs_dep_time % 100
minutes_timedelta = pd.to_timedelta(minutes, unit='m')

# Apply the timedeltas to offset the dates by the departure time
departure_timestamp = date + hours_timedelta + minutes_timedelta
departure_timestamp

0   1993-01-29 10:55:00
1   1993-01-30 10:55:00
2   1993-01-31 10:55:00
3   1993-01-03 17:29:00
4   1993-01-04 17:29:00
5   1993-01-05 17:29:00
6   1993-01-06 17:29:00
7   1993-01-07 17:29:00
8   1993-01-08 17:29:00
9   1993-01-10 17:29:00
dtype: datetime64[ns]

In [5]:
hours = df.CRSDepTime // 100
# hours_timedelta = pd.to_timedelta(hours, unit='h')
hours_timedelta = hours.map_partitions(pd.to_timedelta, unit='h')

minutes = df.CRSDepTime % 100
# minutes_timedelta = pd.to_timedelta(minutes, unit='m')
minutes_timedelta = minutes.map_partitions(pd.to_timedelta, unit='m')

departure_timestamp = df.Date + hours_timedelta + minutes_timedelta

In [6]:
departure_timestamp

Dask Series Structure:
npartitions=4
    datetime64[ns]
               ...
               ...
               ...
               ...
dtype: datetime64[ns]
Dask Name: add, 44 tasks

In [7]:
departure_timestamp.head()

0   1993-01-29 10:55:00
1   1993-01-30 10:55:00
2   1993-01-31 10:55:00
3   1993-01-03 17:29:00
4   1993-01-04 17:29:00
dtype: datetime64[ns]

### Exercise1: 
Rewrite above to use a single call to map_partitions

In [9]:
def compute_departure_timestamp(df):
    #write your code here 
    hours = df.CRSDepTime // 100
    hours_timedelta = pd.to_timedelta(hours, unit='h')

    minutes = df.CRSDepTime % 100
    minutes_timedelta = pd.to_timedelta(minutes, unit='m')

    return df.Date + hours_timedelta + minutes_timedelta

    departure_timestamp = df.map_partitions(compute_departure_timestamp)

In [10]:
departure_timestamp = df.map_partitions(compute_departure_timestamp)


In [11]:
departure_timestamp.head()

0   1993-01-29 10:55:00
1   1993-01-30 10:55:00
2   1993-01-31 10:55:00
3   1993-01-03 17:29:00
4   1993-01-04 17:29:00
dtype: datetime64[ns]